In [34]:
import torch
from torchvision import transforms
from PIL import Image
from transformers import DistilBertTokenizer

# Force reload model module (in case of cached old version)
import importlib
import model as model_module
importlib.reload(model_module)
from model import ViralPredictor

# --- KONFIGURACJA ---
MODEL_PATH = "data/models/viral_predictor.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Definicja transformacji (MUSZĄ być takie same jak w treningu)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# 2. Funkcja ładowania modelu
def load_model():
    print(f"Ładowanie modelu z {MODEL_PATH}...")
    model = ViralPredictor()
    # Wczytanie wag (state_dict)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()  # WAŻNE: Przełącza model w tryb testowy (wyłącza Dropout itp.)
    return model


# 3. Funkcja przewidująca (Inference)
def predict_viral(model, image_path, title_text):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    # A. Przygotowanie Obrazu
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        return "Błąd: Nie znaleziono pliku obrazu."

    # Dodajemy wymiar batcha (Model oczekuje [1, 3, 224, 224], a nie [3, 224, 224])
    image_tensor = transform(image).unsqueeze(0).to(DEVICE)

    # B. Przygotowanie Tekstu
    text_inputs = tokenizer(
        title_text,
        return_tensors="pt",
        padding='max_length',
        truncation=True,
        max_length=50
    )
    input_ids = text_inputs['input_ids'].to(DEVICE)
    attention_mask = text_inputs['attention_mask'].to(DEVICE)

    # C. Predykcja
    with torch.no_grad():  # Wyłączamy obliczanie gradientów (oszczędność pamięci)
        prediction = model(image_tensor, input_ids, attention_mask)

    # Wyciągamy liczbę z tensorów
    score = prediction.item()
    return score


def print_verdict(score, title, image_path):
    """Helper function to print prediction results."""
    print(f"\nTytuł: {title}")
    print(f"Plik: {image_path}")
    print(f"Viral Probability: {score:.2%}")

    if score > 0.6:
        print(">> WERDYKT: 🔥 POTENCJALNY VIRAL!")
    elif score < 0.4:
        print(">> WERDYKT: 💀 SŁABIZNA...")
    else:
        print(">> WERDYKT: 😐 Przeciętniak")



In [39]:
model = load_model()

print("-" * 30)
print("TESTER VIRALOWOŚCI")
print("-" * 30)

# Test 1
test_image = "data/test/test1.jpg"
test_title = "МОРГЕНШТЕРН — Группа Крови (Live in Poland 2025) Полная версия"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)



Tytuł: МОРГЕНШТЕРН — Группа Крови (Live in Poland 2025) Полная версия
Plik: test/test1.jpg
Viral Probability: 50.91%
>> WERDYKT: 😐 Przeciętniak


In [40]:
# Test 2
test_image = "data/test/test2.jpg"
test_title = "How I Transformed VS Code Into a Cozy Workspace #devsetup"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)



Tytuł: How I Transformed VS Code Into a Cozy Workspace #devsetup
Plik: test/test2.jpg
Viral Probability: 84.01%
>> WERDYKT: 🔥 POTENCJALNY VIRAL!


In [41]:
# Test 3
test_image = "data/test/test3.jpg"
test_title = "How I Code Profitable Apps SOLO (Idea + Build + Marketing Guide)"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)



Tytuł: How I Code Profitable Apps SOLO (Idea + Build + Marketing Guide)
Plik: test/test3.jpg
Viral Probability: 50.43%
>> WERDYKT: 😐 Przeciętniak


In [44]:
# Test 4
test_image = "data/test/test4.jpeg"
test_title = 'Michael Jackson\'s Drummer Jonathan Moffett Performs "Smooth Criminal"'
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)



Tytuł: Michael Jackson's Drummer Jonathan Moffett Performs "Smooth Criminal"
Plik: test/test4.jpeg
Viral Probability: 47.46%
>> WERDYKT: 😐 Przeciętniak
